In [1]:
pip install datasets evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and pl

In [2]:
import pandas as pd
import numpy as np
from datasets import Dataset
import evaluate
import torch
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification
)
label2id = {"negative": 0, "neutral": 1, "positive": 2}
id2label = {0: "negative", 1: "neutral", 2: "positive"}

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/train.csv")
val_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/val.csv")
test_df = pd.read_csv("/content/drive/MyDrive/Newton/CzechRestaurants/data/test.csv")

In [5]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("fav-kky/FERNET-C5-RoBERTa")

def tokenize_function(example):
    question_string = f"Jaký je sentiment tohoto aspektu: {example['aspect']}?"

    encoding = tokenizer(
        text=example["text"],
        text_pair=question_string,
        truncation=True,
        padding="max_length",
        max_length=256
    )
    # Convert label from string to int
    encoding["label"] = label2id[example["label"]]
    return encoding

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/932k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/592k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [7]:
train_dataset = train_dataset.map(tokenize_function, batched=False)
val_dataset = val_dataset.map(tokenize_function, batched=False)
test_dataset = test_dataset.map(tokenize_function, batched=False)

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/329 [00:00<?, ? examples/s]

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    "fav-kky/FERNET-C5-RoBERTa",
    num_labels=3
)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at fav-kky/FERNET-C5-RoBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [10]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=200,
    warmup_ratio=0.06,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [11]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fialavi2 (fialavi2-czech-technical-university-in-prague) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.565865,0.775915
2,0.743000,0.546224,0.783537
3,0.461600,0.540551,0.804878
4,0.461600,0.610133,0.812500
5,0.276100,0.679644,0.798780
6,0.181900,0.827569,0.814024
7,0.110900,0.952070,0.817073
8,0.110900,1.021396,0.823171
9,0.067600,1.050234,0.823171
10,0.034600,1.060640,0.820122


TrainOutput(global_step=1440, training_loss=0.26143033454815545, metrics={'train_runtime': 1368.5094, 'train_samples_per_second': 16.777, 'train_steps_per_second': 1.052, 'total_flos': 3020542035517440.0, 'train_loss': 0.26143033454815545, 'epoch': 10.0})

In [12]:
val_results = trainer.evaluate()
print(val_results)

{'eval_loss': 1.0213961601257324, 'eval_accuracy': 0.823170731707317, 'eval_runtime': 7.3353, 'eval_samples_per_second': 89.431, 'eval_steps_per_second': 5.589, 'epoch': 10.0}


In [13]:
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.2206580638885498, 'eval_accuracy': 0.7689969604863222, 'eval_runtime': 3.7325, 'eval_samples_per_second': 88.144, 'eval_steps_per_second': 5.626, 'epoch': 10.0}


In [14]:
trainer.save_model("/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA")

In [15]:
tokenizer.save_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA")

('/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/tokenizer_config.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/special_tokens_map.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/vocab.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/merges.txt',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/added_tokens.json',
 '/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA/tokenizer.json')

In [24]:
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Newton/CzechRestaurants/models/FERNET-C5-RoBERTaQA")

## Vyhodnocovací metriky

In [17]:
def compute_metrics_accuracy(eval_pred):
    metric = evaluate.load("accuracy")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer_accuracy = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_accuracy
)

In [18]:
test_results_accuracy = trainer_accuracy.evaluate()
print(test_results_accuracy)

{'eval_loss': 1.2206579446792603, 'eval_model_preparation_time': 0.0035, 'eval_accuracy': 0.7689969604863222, 'eval_runtime': 4.3978, 'eval_samples_per_second': 74.811, 'eval_steps_per_second': 9.55}


In [19]:
def compute_metrics_weightedf1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="weighted")

trainer_weightedf1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_weightedf1
)

In [23]:
test_results_weightedf1 = trainer_weightedf1.evaluate()
print(test_results_weightedf1)

{'eval_loss': 1.2206579446792603, 'eval_model_preparation_time': 0.0029, 'eval_f1': 0.7670799441716095, 'eval_runtime': 5.007, 'eval_samples_per_second': 65.708, 'eval_steps_per_second': 8.388}


In [21]:
def compute_metrics_macrof1(eval_pred):
    metric = evaluate.load("f1")
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, average="macro")

trainer_macrof1 = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics_macrof1
)

In [22]:
test_results_macrof1 = trainer_macrof1.evaluate()
print(test_results_macrof1)

{'eval_loss': 1.2206579446792603, 'eval_model_preparation_time': 0.0072, 'eval_f1': 0.7580249422354685, 'eval_runtime': 4.4532, 'eval_samples_per_second': 73.88, 'eval_steps_per_second': 9.432}


## Praktické testy

In [25]:
def predict_sentiment(model, tokenizer, aspect, text):
    question_string = f"Jaký je sentiment tohoto aspektu: {aspect}?"

    encoding = tokenizer(
        text=text,
        text_pair=question_string,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=256
    )

    with torch.no_grad():
        outputs = model(**encoding)

    logits = outputs.logits
    print(logits)
    predicted_class_id = logits.argmax(dim=1).item()
    return id2label[predicted_class_id]

### Sada L

In [26]:
example_aspect = "výkon"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L1: {prediction}, expected: positive")
example_aspect = "displej"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L2: {prediction}, expected: negative")
example_aspect = "baterie"
example_text = "Výkon je dobrý, ale displej je rozmazaný a tmavý. Baterie byla skutečným překvapením, nemusím ji přes den nabíjet!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for L3: {prediction}, expected: positive")

tensor([[-2.7039, -1.5700,  5.2299]])
Predicted sentiment for L1: positive, expected: positive
tensor([[ 4.9189, -3.3900, -1.8851]])
Predicted sentiment for L2: negative, expected: negative
tensor([[-2.2784, -1.7769,  4.9098]])
Predicted sentiment for L3: positive, expected: positive


### Sada R

In [27]:
example_aspect = "personál"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R1: {prediction}, expected: negative")
example_aspect = "stůl"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R2: {prediction}, expected: negative")
example_aspect = "jídlo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R3: {prediction}, expected: positive")
example_aspect = "pivo"
example_text = "Když jsme vešli do restaurace, personál byl velmi nepříjemný a náš stůl nebyl uklizený. Na druhou stranu bylo jídlo opravdu chutné a pivo bylo vynikající."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for R4: {prediction}, expected: positive")

tensor([[ 5.0840, -3.1988, -2.2980]])
Predicted sentiment for R1: negative, expected: negative
tensor([[ 2.1228, -1.6341, -0.1976]])
Predicted sentiment for R2: negative, expected: negative
tensor([[-2.9235, -1.5553,  5.4488]])
Predicted sentiment for R3: positive, expected: positive
tensor([[-2.9742, -1.4751,  5.4585]])
Predicted sentiment for R4: positive, expected: positive


### Sada M

In [28]:
example_aspect = "Ósace"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M1: {prediction}, expected: neutral")
example_aspect = "Saremi"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M2: {prediction}, expected: positive")
example_aspect = "Babtel"
example_text = "Na výstavě EXPO 2025 v Ósace představilo mnoho firem své nejnovější produkty. Společnost Saremi úspěšně uvedla nový notebook, který si návštěvníci oblíbili, zatímco nový telefon firmy Babtel byl katastrofa, vůbec nefungoval!"
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for M3: {prediction}, expected: negative")

tensor([[-2.8800,  3.5400, -0.5244]])
Predicted sentiment for M1: neutral, expected: neutral
tensor([[-2.8333,  0.2087,  3.9448]])
Predicted sentiment for M2: positive, expected: positive
tensor([[ 5.0778, -2.7978, -2.6019]])
Predicted sentiment for M3: negative, expected: negative


### Sada O

In [29]:
example_aspect = "den"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O1: {prediction}, expected: positive")
example_aspect = "počasí"
example_text = "Dneska byl opravdu skvělý den, Anička mě pozvala na rande! Bohužel počasí bylo hrozné, takže jsme to museli zrušit."
prediction = predict_sentiment(model, tokenizer, example_aspect, example_text)
print(f"Predicted sentiment for O2: {prediction}, expected: negative")

tensor([[-2.8346, -1.4726,  5.3092]])
Predicted sentiment for O1: positive, expected: positive
tensor([[ 5.0354, -3.2118, -2.0907]])
Predicted sentiment for O2: negative, expected: negative
